# Praca domowa 3 
#### Bartosz Sawicki


In [ ]:
import pandas as pd

## Wczytanie zbioru danych

In [ ]:
path = '../../australia.csv'

df = pd.read_csv(path)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

## Podział na zbiór testowy i treningowy

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop('RainTomorrow', axis='columns')
y = df.loc[:,'RainTomorrow']
display(X,y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1337, 
    stratify=y) # dzielimy tak, żeby zbiór treningowy i testowy miały taki sam rozkład y

## Nauczenie klasyfikatorów
### Regresja logistyczna z l2

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression(penalty='l2', random_state=1337).fit(X_train, y_train)

Przy domyślnym `max_iter`=100 regresja nie jest zbieżna.

In [ ]:
lr = LogisticRegression(penalty='l2', random_state=1337, max_iter=1000).fit(X_train, y_train)

Po zwiększeniu parametru mamy zbieżność.

### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
ab = AdaBoostClassifier(n_estimators=200, random_state=1337).fit(X_train, y_train)

### KNN Clasifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7, p=1).fit(X_train, y_train)

## Ewaluacja modeli

Używane metryki:
- accuracy
- balanced_accuracy
- f1
- roc_auc


In [ ]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score

In [ ]:
models = [lr,ab,knn]
metrics = [accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score]

result_dict = {metric.__name__: {} for metric in metrics}

for model in models:
    y_pred = model.predict(X_test)
    for metric in metrics:
        result_dict.get(metric.__name__).update({type(model).__name__ : metric(y_test, y_pred)})

In [ ]:
metric_names = [metric.__name__ for metric in metrics]
res = pd.DataFrame(result_dict)
res = pd.concat((res,res.index.to_series()), axis=1)
results = res.melt(0,metric_names)
results['metric'] = results['variable']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.catplot(data=results, hue=0, y='value', x='metric', kind='bar', col='metric', sharex=False, col_wrap=2)
plt.show()

Niezależnie, czy patrzymy na wynik jednej metryki, czy na wszystkie, najlepszym klasyfikatorem okazuje się AdaBoost. Niemniej jego przewaga nad innymi modelami jest niewielka.